In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import praw
from datetime import datetime, timezone

In [ ]:
import utils
import tabledefinitions

In [ ]:
cfg_file = utils.findConfig()
cfg = utils.parseConfig(cfg_file)

In [ ]:
CLIENT_ID = cfg['CLIENTID']
CLIENT_SECRET = cfg['CLIENTSECRET']
PASSWORD = cfg['PASSWORD']
USERNAME = cfg['USERNAME']

In [ ]:
reddit = praw.Reddit(
    client_id=f"{CLIENT_ID}",
    client_secret=f"{CLIENT_SECRET}",
    password=f"{PASSWORD}",
    user_agent=f"Post Extraction (by u/{USERNAME})",
    username=f"{USERNAME}",
)

print(reddit.read_only)
#print(reddit.user.me())

Get Rising Reddit Data

In [ ]:
subreddit = "pics"
risingSchema=tabledefinitions.risingTableDefinition.risingSchema
topN=25
view='rising'
risingData = utils.getRedditData(reddit=reddit, subreddit=subreddit, view=view, schema=risingSchema, topN=topN) 

In [ ]:
# for visualization purposes, don't need pandas later
import pandas as pd
pd.DataFrame(risingData)

Push to Dynamo

Steps

1. Set up [IAM Identity Center](https://docs.aws.amazon.com/singlesignon/latest/userguide/getting-started.html). Create a permission set and a user under this permission set. Set up MFA with this user.
2. Install [AWS CLI V2](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html)
3. [Install Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#install-boto3) and [set up the config file](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html#aws-iam-identity-center) for the IAM Identity Center. During the set up you will be prompted to log into the user you created with the MFA method you previously set up.

In [ ]:
import boto3

session = boto3.Session(profile_name='AdministratorAccess', region_name='us-east-2')
# resource vs client: https://www.learnaws.org/2021/02/24/boto3-resource-client/
dynamodb_resource = session.resource('dynamodb')  #  higher level abstractions, recommended to use, fewer methods but creating table returns a table object that you can run operations on, can also grab a Table with Table('name')
# dynamodb_client = session.client('dynamodb')  # low-level, more explicit methods. Creating table returns a dictionary

In [ ]:
%%time
tableName = tabledefinitions.risingTableDefinition.tableName  # table for collecting rising posts data
risingRawTableDefinition = tabledefinitions.risingTableDefinition.risingRawTableDefinition
risingSchema = tabledefinitions.risingTableDefinition.risingSchema

risingTable = utils.getOrCreateTable(risingRawTableDefinition, dynamodb_resource)

In [ ]:
utils.batchWriter(risingTable, risingData, risingSchema)

To do:

- write a function that gathers the top data
- write a function that cleans up data older than ~24 hours
- write a function that cleans the current dynamo data and joins it to targets (the top data)

In [ ]:
# currently this is basically the same as rising but I wanted to make this framework flexible
# enough to accept different forms of the data
hotSchema=tabledefinitions.hotTableDefinition.hotSchema
topN=3
view='hot'
hotData = utils.getRedditData(reddit=reddit, subreddit=subreddit, view=view, schema=hotSchema, topN=topN) 